In [1]:
import requests
import json
import pandas as pd
import plotly.graph_objects as go
import datetime
import coordinator

In [2]:
config_file = "config.json"
with open(config_file, "r") as input_file:
    config = json.load(input_file)

In [3]:
# query 24 hours
start_timestamp = 1692619200 # unix epoch in utc
interval = 900 # timestep in seconds (= 15 minutes)
number_intervals = 12
query_times = [start_timestamp + i*interval for i in range(0, number_intervals)]

In [4]:
consumers = coordinator.query_consumers(config['consumer_api'])
network = coordinator.query_network(config['network_api'])
generators = coordinator.query_generators(config['generator_api'])
opfs = coordinator.query_range(query_times, config['network_api'])

In [6]:
nodes_df = coordinator.get_nodes(consumers, generators, network)
lines_df = coordinator.get_line_connections(consumers, generators, network)
res_load_ts = coordinator.load_ts(nodes_df, opfs)
res_bus_ts = coordinator.bus_ts(nodes_df, opfs)
res_ext_grid_ts = coordinator.ext_grid_ts(nodes_df, opfs)
res_gen_ts = coordinator.gen_ts(nodes_df, opfs)
res_sum_load_ts = res_load_ts.groupby(by=['p_mw', 'q_mvar', 'time']).sum()

In [ ]:
nodes_df

,name,identifier,latitude,longitude,category,type
0,location 1,e6c8200f-d84b-499b-ad13-4313eec39ca2,48.3744,14.5123,household,load
1,location 2,60d36bfa-92a2-4521-95fa-ed0e95f2413f,48.3744,14.5115,household,load
2,location 3,ec28306c-772c-4d65-9916-55c03ce61566,48.3740,14.5111,household,load
3,generator 1,8bee677b-b929-4d52-ba2d-d8619b86e199,48.3744,14.5125,household,generator
4,grid connection,d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab,48.3744,14.5131,network,network


In [8]:
mapbox_token = ""

fig = go.Figure()

opf_keys = [k for k in sorted(opfs.keys())]
lines_df = coordinator.get_line_connections(consumers, generators, network)
load_df = coordinator.create_load_dataframe(consumers, network, opfs[opf_keys[0]])
load_map_df = nodes_df[nodes_df['type'] == "load"]
generator_map_df = nodes_df[nodes_df['type'] == "generator"]
network_map_df = nodes_df[nodes_df['type'] == "network"]

# add lines
fig.add_trace(go.Scattermapbox(
    mode = 'lines',
    lon = lines_df['longitude'],
    lat = lines_df['latitude']
))

# nodes
fig.add_trace(go.Scattermapbox(
    mode = "markers",
    lon = load_map_df['longitude'],
    lat = load_map_df['latitude'],
    marker = go.scattermapbox.Marker(
        color="red"
    ),
    hoverinfo = 'text',
    text = load_map_df['name'] # + '<br>' + 'load: ' + load_map_df['load'].astype(str) + " MW"
))

fig.add_trace(go.Scattermapbox(
    mode = "markers",
    lon = network_map_df['longitude'],
    lat = network_map_df['latitude'],
    marker = go.scattermapbox.Marker(
        color="blue"
    ),
    hoverinfo = 'text',
    text = network_map_df['name'] # + '<br>' + 'load: ' + load_map_df['load'].astype(str) + " MW"
))

fig.add_trace(go.Scattermapbox(
    mode = "markers",
    lon = generator_map_df['longitude'],
    lat = generator_map_df['latitude'],
    marker = go.scattermapbox.Marker(
        color="green"
    ),
    hoverinfo = 'text',
    text = generator_map_df['name'] # + '<br>' + 'load: ' + load_map_df['load'].astype(str) + " MW"
))

fig.update_layout(    
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox_style = "dark",
    mapbox = {
        'center': { 'lon': nodes_df['longitude'].mean(), 'lat': nodes_df['latitude'].mean() },
        'zoom': 15,
        'style': "open-street-map"})

#fig.update_geos(projection_type='mercator')

fig.show()

In [10]:
nodes_df

,name,identifier,latitude,longitude,category
0,location 1,e6c8200f-d84b-499b-ad13-4313eec39ca2,48.3744,14.5123,household
1,location 2,60d36bfa-92a2-4521-95fa-ed0e95f2413f,48.3744,14.5115,household
2,location 3,ec28306c-772c-4d65-9916-55c03ce61566,48.3740,14.5111,household
3,generator 1,8bee677b-b929-4d52-ba2d-d8619b86e199,48.3744,14.5125,household
4,grid connection,d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab,48.3744,14.5131,network


In [246]:
def line_ts(nodes_df, opfs):
    node_identifier_map = create_node_identifier_mapping(nodes_df)
    opf_keys = [k for k in sorted(opfs.keys())]
    dataframes = []
    for ok in opf_keys:
        o = opfs[ok]
        df1 = pd.DataFrame(o['res_line'])
        df1 = df1.reset_index()
        df1['node_id'] = df1['index'].astype('int64')
        dataframe = df1
        dataframe['time'] = datetime.datetime.fromtimestamp(ok)
        dataframes.append(dataframe)
    res_line_ts = pd.concat(dataframes).reset_index()#[['node_id', 'p_mw', 'q_mvar', 'time']]
    #res_line_ts['identifier'] = res_line_ts['node_id'].map(node_identifier_map)
    #res_line_ts = res_line_ts[['p_mw', 'q_mvar', 'time', 'identifier']]
    return res_line_ts

In [128]:
res_load_ts = load_ts(nodes_df, opfs)
res_sum_load_ts = res_load_ts[['time', 'p_mw', 'q_mvar']].groupby(by=['time']).sum().reset_index().sort_values(by=['time'])

res_gen_ts = gen_ts(nodes_df, opfs)

fig_sum_load = go.Figure()

res_ext_grid_ts = ext_grid_ts(nodes_df, opfs)

fig_sum_load.add_trace(go.Scatter(x=res_sum_load_ts['time'], y=res_sum_load_ts['p_mw'],
                mode='markers+lines',
                name='consumer load'))


fig_sum_load.add_trace(go.Scatter(x=res_ext_grid_ts['time'], y=res_ext_grid_ts['p_mw'],
                mode='markers+lines',
                name='grid load'))

fig_sum_load.add_trace(go.Scatter(x=res_gen_ts['time'], y=res_gen_ts['p_mw'],
                mode='markers+lines',
                name='production'))

#fig_sum_load.add_trag
fig_sum_load.show()